<h1>Resource Estimate Reporting Schema</h1>

<font size="3">

<hr>
DISTRIBUTION STATEMENT A. Approved for public release. Distribution is unlimited.

This material is based upon work supported by the Under Secretary of Defense for
Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions,
findings, conclusions or recommendations expressed in this material are those of the
author(s) and do not necessarily reflect the views of the Under Secretary of Defense
for Research and Engineering.

© 2023 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part
252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government
rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed
above. Use of this work other than as specifically authorized by the U.S. Government
may violate any copyrights that exist in this work.
<hr><hr>
</font>

In [1]:
import numpy              as  np
import pandas             as  pd
import os                 as  os

import pprint 

import matplotlib         as  mpl
import matplotlib.pyplot  as  plt

import json               as  json
import jsonschema         as  schema


pd.set_option('display.max_rows',    1000)
pd.set_option('display.max_columns', 1000)

pp = pprint.PrettyPrinter(indent=4)

In [2]:
from resource_reporting   import    schema_resource_estimate
from resource_reporting   import    load_nested_json

<h4><b>Estimate Definition<b></h4>

Example estimate dictionary, listing both required and optional inputs. Note that "number" can be either a real number or an integer, to allow for cases where data represents an ensemble average (e.g., over a set of molecules with the same scale but different connectivity)

In [3]:
fermi_hubbard_estimate = {
    "id":                    "MITLL FH Demo 1",                   # Unique Identifier (REQUIRED, string)
    "name":                  "Fermi-Hubbard",                     # Application name (REQUIRED, string)
    "category":              "scientific",                        # Category: either "scientific" or "industrial" (REQUIRED)
    "size":                  "128 x 128",                         # Description of instance size (problem specific) (REQUIRED, string)
    "task":                  "time_independent_dynamics",         # Computational task. Options: (REQUIRED)
                                                                  #          "ground_state_energy_estimation"
                                                                  #          "time_independent_dynamics"
                                                                  #          "time_dependent_dynamics"
                                                                  #          "linear_system"
                                                                  #          "nonlinear_differential_equation"
    "implementation":        "QET w/ Linear-T FH Encoding",       # Description of implementation, e.g. algorithm / encoding (REQUIRED, string)
    "value":                 50e6,                                # Financial value (dollars)        (REQUIRED, number)
    "value_ci":              [50e5, 50e7],                        # Confidence interval bounds for value (dollars)    (optinal, number)
    "value_per_t_gate":      5.0e-4,                              # value per T-gate, including all repetitions required
                                                                  # for utility (see below)
    "circuit_repetitions_per_calculation":      100,              # Circuit repetitions required for successful execution
                                                                  # of the algorithm, ex: a GSEE run   (REQUIRED, number)
    "calculation_repetitions":                  10,               # Calculation repetitions required for utility, e.g.,
                                                                  # parameter sweeps (REQUIRED, number)
    "total_circuit_repetitions":                1000,             # Total number of circuitrepetitions required
                                                                  # to meet utlity target (REQUIRED, number)
    "runtime_requirement":                      115200,           # Maximum runtime for utility, assuming a single, 
                                                                  # unparallelized workflow (seconds; REQUIRED, number)
    "logical-abstract":                                           # ABSTRACT LOGICAL ESTIMATES (REQUIRED) 
    {  
      "num_qubits":     32804,                                    # Number of logical qubits (REQUIRED, number)
      "t_count":        60549453290,                              # Total T-gate count (REQUIRED, number)
      "clifford_count": 310704519223,                             # Total clifford count (optional, number)
      "gate_count":     99999999,                                 # Total gate count (optional, number)
      "circuit_depth":  99999999,                                 # Total circuit depth (optional, number)
      "t_depth":        99999999                                  # Total T-depth (optional, number)
    },

    "logical-compiled":                                           # COMPILED LOGICAL ESTIMATES (optional) 
    {
          "logical_architecture_description":     "Description",  # Description of logical architecture (REQUIRED, string)
          "num_qubits":                           99999999,       # Number of logical qubits (REQUIRED, number)
          "t_count":                              99999999,       # Total T-gate count (REQUIRED, number)
          "num_t_factories":                      10,             # Number of T-factories (REQUIRED, integer)
          "gate_count":                           99999999,       # Total gate count (optional, number)
          "clifford_count":                       99999999,       # Total clifford count (optional, number)
          "circuit_depth":                        99999999,       # Total circuit depth (optional, number)
          "t_depth":                              99999999        # Total T-depth (optional, number)
    },   

    "physical":                                                   # PHYSICAL RESOURCE ESTIMATES (optional) 
    {
          "physical_architecture_description":    "Description",  # Description of physical architecture (REQUIRED, string)
          "code_name":                            "surface",      # Type of code: "surface" or "other"  (REQUIRED)
          "code_distance":                        8,              # Code distance (REQUIRED, integer)
          "runtime":                              99999999,       # Runtime in seconds (REQUIRED, number)
          "num_qubits":                           99999999,       # Number of physical qubits (REQUIRED, number)
          "t_count":                              99999999,       # Total T-gate count  (REQUIRED, number)
          "num_t_factories":                      10,             # Number of T-factories (REQUIRED, integer)
          "num_factory_qubits":                   100,            # Number physical qubits used for T-factories (optional, integer)
          "gate_count":                           99999999,       # Total gate count (optional, number)
          "circuit_depth":                        99999999,       # Total circuit depth (optional, number)
          "t_depth":                              99999999,       # Total T-depth (optional, number)
          "clifford_count":                       99999999        # Total clifford count (optional, number)
    }
}


Validate entry to ensure compliance with reporting schema:

In [4]:
schema.validate( instance=fermi_hubbard_estimate,
                 schema=schema_resource_estimate  )

<h4><b><u>IO Operations</u><b></h4>

Write dict to json file:

In [5]:
filename        =  "../data/fh_qsvt-128x1280-demo.json"
estimate_data   =  fermi_hubbard_estimate

In [6]:
with open(filename, 'w+') as fp:
    json.dump(estimate_data, fp, indent=2)

Read dict from json file:

In [7]:
with open(filename) as file:
    data = json.load(file)

In [8]:
schema.validate( instance=data,
                 schema=schema_resource_estimate  )

In [9]:
print(json.dumps(data,sort_keys=False, indent=4))

{
    "id": "MITLL FH Demo 1",
    "name": "Fermi-Hubbard",
    "category": "scientific",
    "size": "128 x 128",
    "task": "time_independent_dynamics",
    "implementation": "QET w/ Linear-T FH Encoding",
    "value": 50000000.0,
    "value_ci": [
        5000000.0,
        500000000.0
    ],
    "value_per_t_gate": 0.0005,
    "circuit_repetitions_per_calculation": 100,
    "calculation_repetitions": 10,
    "total_circuit_repetitions": 1000,
    "runtime_requirement": 115200,
    "logical-abstract": {
        "num_qubits": 32804,
        "t_count": 60549453290,
        "clifford_count": 310704519223,
        "gate_count": 99999999,
        "circuit_depth": 99999999,
        "t_depth": 99999999
    },
    "logical-compiled": {
        "logical_architecture_description": "Description",
        "num_qubits": 99999999,
        "t_count": 99999999,
        "num_t_factories": 10,
        "gate_count": 99999999,
        "clifford_count": 99999999,
        "circuit_depth": 99999999,
    

<h4><b><u>Dataframe Operations</u></b></h4>

Reporting datasets can be readily slurped up into a dataframe:

In [14]:

df_all = pd.DataFrame()

base_dir = "./demo_estimates/"

for filename in os.listdir(base_dir):
    print(filename)
    df_data = load_nested_json(base_dir+filename,label=filename)
    # print(df_data)
    if df_all.empty:
        df_all = df_data
    else:
        df_all = pd.concat([df_all, df_data],axis=1)
        

fh-qet-64x64.dat
fh-qet-32x32.dat
fh-qet-8x8.dat
fh-qet-2x2.dat
fh-qet-4x4.dat
fh-qet-16x16.dat
fh-qet-128x128.dat


In [15]:
df_all

,fh-qet-64x64.dat,fh-qet-32x32.dat,fh-qet-8x8.dat,fh-qet-2x2.dat,fh-qet-4x4.dat,fh-qet-16x16.dat,fh-qet-128x128.dat
id,FH 64x64 Square (U/J = 4.0),FH 32x32 Square (U/J = 4.0),FH 8x8 Square (U/J = 4.0),FH 2x2 Square (U/J = 4.0),FH 4x4 Square (U/J = 4.0),FH 16x16 Square (U/J = 4.0),FH 128x128 Square (U/J = 4.0)
name,Fermi-Hubbard,Fermi-Hubbard,Fermi-Hubbard,Fermi-Hubbard,Fermi-Hubbard,Fermi-Hubbard,Fermi-Hubbard
category,scientific,scientific,scientific,scientific,scientific,scientific,scientific
size,64 x 64,32 x 32,8 x 8,2 x 2,4 x 4,16 x 16,128 x 128
task,time_independent_dynamics,time_independent_dynamics,time_independent_dynamics,time_independent_dynamics,time_independent_dynamics,time_independent_dynamics,time_independent_dynamics
implementation,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding,QET w/ Linear-T FH Encoding
value,50000000.0,50000000.0,50000000.0,50000000.0,50000000.0,50000000.0,50000000.0
value_ci,"[5000000.0, 500000000.0]","[5000000.0, 500000000.0]","[5000000.0, 500000000.0]","[5000000.0, 500000000.0]","[5000000.0, 500000000.0]","[5000000.0, 500000000.0]","[5000000.0, 500000000.0]"
value_per_t_gate,0.000013,0.000013,0.000013,0.591,0.144,0.000013,0.000001
circuit_repetitions_per_calculation,100,100,100,100,100,100,100


In [16]:
df_all.loc[['size','t_count','num_qubits']]

,fh-qet-64x64.dat,fh-qet-32x32.dat,fh-qet-8x8.dat,fh-qet-2x2.dat,fh-qet-4x4.dat,fh-qet-16x16.dat,fh-qet-128x128.dat
size,64 x 64,32 x 32,8 x 8,2 x 2,4 x 4,16 x 16,128 x 128
t_count,3855739870,257575242,2135722,84650,346206,19909982,60549453290
num_qubits,8224,2076,148,20,48,536,32804
